In [8]:
#we use the findspark library to locate spark on our local machine
import findspark
findspark.init(r'C:\spark\spark-3.5.0-bin-hadoop3')
import pyspark # only run this after findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('pyspark-by-examples').getOrCreate()

arrayData = [
        ('James',['Java','Scala'],{'hair':'black','eye':'brown'}),
        ('Michael',['Spark','Java',None],{'hair':'brown','eye':None}),
        ('Robert',['CSharp',''],{'hair':'red','eye':''}),
        ('Washington',None,None),
        ('Jefferson',['1','2'],{})
        ]
df = spark.createDataFrame(data=arrayData, schema = ['name','knownLanguages','properties'])
df.printSchema()
df.show()

root
 |-- name: string (nullable = true)
 |-- knownLanguages: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

+----------+-------------------+--------------------+
|      name|     knownLanguages|          properties|
+----------+-------------------+--------------------+
|     James|      [Java, Scala]|{eye -> brown, ha...|
|   Michael|[Spark, Java, NULL]|{eye -> NULL, hai...|
|    Robert|         [CSharp, ]|{eye -> , hair ->...|
|Washington|               NULL|                NULL|
| Jefferson|             [1, 2]|                  {}|
+----------+-------------------+--------------------+



In [9]:
from pyspark.sql.functions import explode
df2 = df.select(df.name,explode(df.knownLanguages))
df2.printSchema()
df2.show()

root
 |-- name: string (nullable = true)
 |-- col: string (nullable = true)

+---------+------+
|     name|   col|
+---------+------+
|    James|  Java|
|    James| Scala|
|  Michael| Spark|
|  Michael|  Java|
|  Michael|  NULL|
|   Robert|CSharp|
|   Robert|      |
|Jefferson|     1|
|Jefferson|     2|
+---------+------+



In [10]:
from pyspark.sql.functions import explode
df3 = df.select(df.name,explode(df.properties))
df3.printSchema()
df3.show()

root
 |-- name: string (nullable = true)
 |-- key: string (nullable = false)
 |-- value: string (nullable = true)

+-------+----+-----+
|   name| key|value|
+-------+----+-----+
|  James| eye|brown|
|  James|hair|black|
|Michael| eye| NULL|
|Michael|hair|brown|
| Robert| eye|     |
| Robert|hair|  red|
+-------+----+-----+



df.select(...): This selects one or more columns from a DataFrame (df). It returns a new DataFrame containing only the specified columns.

df.name: This selects the column named "name" from the DataFrame df.

explode_outer(df.knownLanguages): This is a PySpark function called explode_outer. It is used to transform a column of arrays or maps into multiple rows, with each row containing a single element from the array or map. In this case, it is applied to the column knownLanguages.

.show(): This is used to display the resulting DataFrame on the console.

Putting it all together, the code takes a DataFrame df, selects the "name" column, and then applies the explode_outer function to the "knownLanguages" column. This will generate a new DataFrame where each row corresponds to a single language known by a person, and if a person knows multiple languages, there will be multiple rows for that person with the same name but different languages. The result is displayed on the console using the show() method.

In [11]:
from pyspark.sql.functions import explode_outer
""" with array """
df.select(df.name,explode_outer(df.knownLanguages)).show()

+----------+------+
|      name|   col|
+----------+------+
|     James|  Java|
|     James| Scala|
|   Michael| Spark|
|   Michael|  Java|
|   Michael|  NULL|
|    Robert|CSharp|
|    Robert|      |
|Washington|  NULL|
| Jefferson|     1|
| Jefferson|     2|
+----------+------+



In [12]:
""" with map """
df.select(df.name,explode_outer(df.properties)).show()

+----------+----+-----+
|      name| key|value|
+----------+----+-----+
|     James| eye|brown|
|     James|hair|black|
|   Michael| eye| NULL|
|   Michael|hair|brown|
|    Robert| eye|     |
|    Robert|hair|  red|
|Washington|NULL| NULL|
| Jefferson|NULL| NULL|
+----------+----+-----+



df.select(...): This selects one or more columns from a DataFrame (df). It returns a new DataFrame containing only the specified columns.

df.name: This selects the column named "name" from the DataFrame df.

posexplode(df.knownLanguages): This is a PySpark function called posexplode. It is used to transform a column of arrays into multiple rows, with each row containing the index (position) and the corresponding element from the array. In this case, it is applied to the column knownLanguages.

.show(): This is used to display the resulting DataFrame on the console.

Putting it all together, the code takes a DataFrame df, selects the "name" column, and then applies the posexplode function to the "knownLanguages" column. This will generate a new DataFrame where each row corresponds to a single element (language) from the "knownLanguages" array along with its position (index) in the array. The result is displayed on the console using the show() method.

In [13]:
from pyspark.sql.functions import posexplode
""" with array """
df.select(df.name,posexplode(df.knownLanguages)).show()

+---------+---+------+
|     name|pos|   col|
+---------+---+------+
|    James|  0|  Java|
|    James|  1| Scala|
|  Michael|  0| Spark|
|  Michael|  1|  Java|
|  Michael|  2|  NULL|
|   Robert|  0|CSharp|
|   Robert|  1|      |
|Jefferson|  0|     1|
|Jefferson|  1|     2|
+---------+---+------+



In [14]:
""" with map """
df.select(df.name,posexplode(df.properties)).show()

+-------+---+----+-----+
|   name|pos| key|value|
+-------+---+----+-----+
|  James|  0| eye|brown|
|  James|  1|hair|black|
|Michael|  0| eye| NULL|
|Michael|  1|hair|brown|
| Robert|  0| eye|     |
| Robert|  1|hair|  red|
+-------+---+----+-----+



In [15]:
from pyspark.sql.functions import posexplode_outer
""" with array """
df.select(df.name,posexplode_outer(df.knownLanguages)).show()

""" with map """
df.select(df.name,posexplode_outer(df.properties)).show()

+----------+----+------+
|      name| pos|   col|
+----------+----+------+
|     James|   0|  Java|
|     James|   1| Scala|
|   Michael|   0| Spark|
|   Michael|   1|  Java|
|   Michael|   2|  NULL|
|    Robert|   0|CSharp|
|    Robert|   1|      |
|Washington|NULL|  NULL|
| Jefferson|   0|     1|
| Jefferson|   1|     2|
+----------+----+------+

+----------+----+----+-----+
|      name| pos| key|value|
+----------+----+----+-----+
|     James|   0| eye|brown|
|     James|   1|hair|black|
|   Michael|   0| eye| NULL|
|   Michael|   1|hair|brown|
|    Robert|   0| eye|     |
|    Robert|   1|hair|  red|
|Washington|NULL|NULL| NULL|
| Jefferson|NULL|NULL| NULL|
+----------+----+----+-----+

